Bayan Nezamabad - 20251971, Euan Bourke - 21332142

In [17]:
# Imports
import matplotlib.pyplot as plt
from keras import utils, Sequential, layers, optimizers

In [18]:
# Hyper Params
EPOCHS = 50
BATCH_SIZE = 128
VERBOSE = 1
OPTIMIZER = optimizers.Adam(learning_rate=0.001)
LOSS = "categorical_crossentropy"
METRICS = ["accuracy", "precision", "recall"]

IMG_ROW, IMG_COL = 227, 227 # Image dimensions
INPUT_SHAPE = (IMG_ROW, IMG_COL, 3) # 3 as RGB
NB_CLASSES = 100 # 100 different classifications for 100 different species of butterfly or moth

TRAIN_PATH = 'data/train'
VALID_PATH = 'data/valid'
TEST_PATH = 'data/test'
SEED = 123 # used for consistent randomisation of the dataset

In [19]:
# Loading and preparing training, validation, and testing data
# Images are originally 224x224 so we have to resize them to 227x227, the input size of AlexNet
train_data = utils.image_dataset_from_directory(TRAIN_PATH,
                                                shuffle=True,
                                                image_size=(IMG_COL, IMG_ROW), # Images are resized to 227 x 227
                                                batch_size=BATCH_SIZE,
                                                seed=SEED,
                                                label_mode="categorical")
valid_data = utils.image_dataset_from_directory(VALID_PATH,
                                                shuffle=True,
                                                image_size=(IMG_COL, IMG_ROW), # Images are resized to 227 x 227
                                                batch_size=BATCH_SIZE,
                                                seed=SEED,
                                                label_mode="categorical")
    
test_data = utils.image_dataset_from_directory(TEST_PATH,
                                               shuffle=True,
                                               image_size=(IMG_COL, IMG_ROW), # Images are resized to 227 x 227
                                               batch_size=BATCH_SIZE,
                                               seed=SEED,
                                               label_mode="categorical")

# Min-max normalisation scales all values to the range 0 - 1
# Division by 255 accomplishes this since the max value for any pixel in any colour channel is 255
train_data = train_data.map(lambda x, y: (x / 255.0, y))
valid_data = valid_data.map(lambda x, y: (x / 255.0, y))
test_data = test_data.map(lambda x, y: (x / 255.0, y))

Found 2593 files belonging to 20 classes.
Found 100 files belonging to 20 classes.
Found 100 files belonging to 20 classes.


In [20]:
# Implementation of AlexNet architecture
class AlexNet(Sequential):
    def __init__(self):
        super().__init__()
        self.build_layers()
        self.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)

    def build_layers(self):
        # The architecture consists of 5 convolutional layers, 3 max pooling layers, 1 flatten layer, 3 fully connected layers (dropout following the first two), and an output layer
        
        # First convolutional layer consists of 96 filters, 11x11 kernel and moves in strides of 4
        # This results in an output shape 55x55x96
        self.add(layers.Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=INPUT_SHAPE))
        self.add(layers.BatchNormalization())
        # Taking the max value of a 3x3 region and moving in strides of 2
        # Resultant shape: 27x27x96
        self.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        # The resultant shape should be 27x27x256 so we apply even padding
        self.add(layers.Conv2D(256, (5, 5), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())
        # Resultant shape: 13x13x256
        self.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        # Three more convolutional layers, only dimension changing here is z
        self.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())
        self.add(layers.Conv2D(384, (3, 3), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())
        self.add(layers.Conv2D(256, (3, 3), padding='same', activation='relu'))
        self.add(layers.BatchNormalization())

        # Last max pooling layer results in 6x6x256
        self.add(layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

        # Flatten layer results in 1x1x9216 (6*6*256 = 9216)
        self.add(layers.Flatten())

        # Fully connected layers with dropout to prevent overfitting
        self.add(layers.Dense(4096, activation='relu'))
        # 50% chance for each neuron to be deactivated
        # Prevents overfitting by preventing over reliance on a specific feature and instead learning a more robust set of features
        self.add(layers.Dropout(rate=0.5))
        
        self.add(layers.Dense(4096, activation='relu'))
        self.add(layers.Dropout(rate=0.5))
        
        self.add(layers.Dense(1000, activation='relu'))
        
        self.add(layers.Dense(NB_CLASSES, activation='softmax'))

model = AlexNet()
model.summary()

Model: "alex_net_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_10          │ (None, 55, 55, 96)     │           384 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_11          │ (None, 27, 27, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 13, 13, 384)    │         1,536 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 13, 13, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1000)           │     4,097,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 20)             │        20,020 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,403,868 (238.05 MB)

 Trainable params: 62,401,116 (238.04 MB)

 Non-trainable params: 2,752 (10.75 KB)

In [ ]:
model_history = model.fit(train_data, epochs=EPOCHS, validation_data=valid_data, verbose=VERBOSE)

In [ ]:
# Obtaining metrics from training
training_accuracy = model_history.history['accuracy']
training_precision = model_history.history['precision']
training_recall = model_history.history['recall']
training_loss = model_history.history['loss']

# Obtaining metrics from validation
validation_accuracy = model_history.history['val_accuracy']
validation_precision = model_history.history['val_precision']
validation_recall = model_history.history['val_recall']
validation_loss = model_history.history['val_loss']

# Plotting model accuracy
plt.plot(training_accuracy)
plt.plot(validation_accuracy)
plt.title("Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epochs")
plt.legend(["Train", "Val"], loc="upper left")
plt.show()

# Plotting model loss
plt.plot(training_loss)
plt.plot(validation_loss)
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Val'], loc='upper right')
plt.show()

# Plotting model precision
plt.plot(training_precision)
plt.plot(validation_precision)
plt.title('Precision')
plt.ylabel('Precision')
plt.xlabel('Epochs')
plt.legend(["Train", "Val"], loc="upper right")
plt.show()

# Plotting model recall
plt.plot(training_recall)
plt.plot(validation_recall)
plt.title('Recall')
plt.ylabel('Recall')
plt.xlabel('Epochs')
plt.legend(["Train", "Val"], loc="upper right")
plt.show()

In [ ]:
model.evaluate(test_data, verbose=1)

Refs: \
AlexNet Architecture: https://www.kaggle.com/code/blurredmachine/alexnet-architecture-a-complete-guide \
https://medium.com/@siddheshb008/alexnet-architecture-explained-b6240c528bd5
Keras.layer docs: https://www.tensorflow.org/api_docs/python/tf/keras/layers/
